Use article as reference: 
https://towardsdatascience.com/how-to-build-and-deploy-a-machine-learning-model-with-fastapi-64c505213857

The article is structured as follows:
1. FastAPI installation and building the first API
2. Interactive documentation exploration
3. Training a machine learning model
4. Building a complete REST API
5. Testing
6. Conclusion

### FastAPI installation and building the first API 

Install lib along with ASGI server: Unicorn and Hypercorn
http://www.uvicorn.org/
https://gitlab.com/pgjones/hypercorn

In [3]:
conda install fastapi

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
conda install uvicorn

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


Make a simple API with two endpoints.
1. Import libraries -- both FastAPI and Uvicorn
2. Create an instance of the FastAPI class
3. Declare the first route -- returns a simple Json object on the index page(http://127.0.0.1:8000--this is configured in step 5)
4. Declare the final route -- returns a simple JSON object containing a personalized message. The name parameter comes directly form the URL (e.g http://127.0.02.:8000/Eric)
5. Run the API with Uvicorn

In [1]:
%%writefile test.py

#1. Import libraries -- both FastAPI and Uvicorn
import uvicorn
from fastapi import FastAPI

#2. Create the app object
app = FastAPI()

#3. Index route, opens automatically  on http://127.0.0.1:8000
@app.get('/')
def index():
    ''' This is a first docstring '''
    return {'message': 'Hello, stranger'}

#4. Route with a single parameter, returns the parameter within a message
# Locate at: http://127.0.0.1:8000/AnyNameHere
@app.get('/{name}')
def get_name(name:str):
    ''' This is a second docstring '''
    return {'message': f'Hello, {name}'}

#5. Run the API with uvicorn
# Will run on http://127.0.0.1:8000
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)

Overwriting test.py


In [2]:
!cat test.py


#1. Import libraries -- both FastAPI and Uvicorn
import uvicorn
from fastapi import FastAPI

#2. Create the app object
app = FastAPI()

#3. Index route, opens automatically  on http://127.0.0.1:8000
@app.get('/')
def index():
    ''' This is a first docstring '''
    return {'message': 'Hello, stranger'}

#4. Route with a single parameter, returns the parameter within a message
# Locate at: http://127.0.0.1:8000/AnyNameHere
@app.get('/{name}')
def get_name(name:str):
    ''' This is a second docstring '''
    return {'message': f'Hello, {name}'}

#5. Run the API with uvicorn
# Will run on http://127.0.0.1:8000
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)


Can view this throu:
 1. http://127.0.0.1:8000 (The welcome index route)
 2. http://127.0.0.1:8000/Eric (What I want it to execute)
 3. http://127.0.0.1:8000/docs 

## Deploying an Iris Classifier ML model

In [1]:
# Download data
import requests
iris_csv_url = "https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv"
req = requests.get(iris_csv_url)
url_content = req.content
iris = open('iris.csv', 'wb')
iris.write(url_content)
iris.close()

In [2]:
%%writefile model.py
# Deploying a model on Iris DataSet
#1. Library imports
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from pydantic import BaseModel
import joblib

#2. Class which describes a single flower measurements
class IrisSpecies(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float
        
#3. Class for training the model and making predictions
class IrisModel: 
    # 6. Class constructor, loads the dataset and loads the model
    # if exists. If not, calls the _train_model method and saves model
    def __init__(self):
        self.df = pd.read_csv('iris.csv')
        self.model_fname_ = 'iris_model.pk1'
        try:
            self.model = joblib.load(self.model_fname_) # trying to load model
        except Exception as _:
            self.model = self._train_model()
            joblib.dump(self.model, self.model_fname_)  # train and load the model if it didnt load
    
    # 4. Perform model training using the RandomForest Classifier
    def _train_model(self):
        X = self.df.drop('species', axis=1)
        y = self.df['species']
        rfc = RandomForestClassifier()
        model = rfc.fit(X,y)
        return model
    
    # 5. Make a prediction based on the user-entered data
    # Returns the predicted species with its respective probability
    def predict_species(self, sepal_length, sepal_width, petal_length, petal_width):
        data_in = [[sepal_length, sepal_width, petal_length, petal_width]]
        prediction = self.model.predict(data_in)
        probability = self.model.predict_proba(data_in).max()
        return prediction[0], probability
      

Overwriting model.py


## Building a complete REST API

In [3]:
%%writefile app.py
# 1. Library imports
import uvicorn
from fastapi import FastAPI
from model import IrisModel, IrisSpecies

# 2. Create app and model objects
app = FastAPI()
model = IrisModel()

# 3. Expose the prediction functionality, make a prediction from the passed 
#  JSON data and return the predicted flower species with the confidence
@app.post('/predict')
def predict_species(iris: IrisSpecies):
    data = iris.dict()
    prediction, probability = model.predict_species(data['sepal_length'], data['sepal_width'],
                                                    data['petal_length'], data['petal_width'])
    return {'prediction': prediction,
            'probability': probability}
#4. Run the API with uvicorn
# Will run on http://127.0.0.1:8000
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)

Overwriting app.py


In [4]:
import requests 

new_measurement = {
    'sepal_length': 5.7,
    'sepal_width': 3.1,
    'petal_length': 4.9,
    'petal_width': 2.2
}

response = requests.post('http://127.0.0.1:8000/predict', json=new_measurement)
print(response.content)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000201F5295BE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))